# Классификация изображений

### Основная идея этого решения: взять предобученую на ImageNet сеть EfficientNet и дообучить под нашу задачу. 


In [ ]:
!nvidia-smi

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import zipfile
import csv
import sys
import os


import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import LearningRateScheduler, ModelCheckpoint
from tensorflow.keras.callbacks import Callback, ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.regularizers import l2
from tensorflow.keras import optimizers

from tensorflow.keras.models import Model
import tensorflow.keras.models as Model
from tensorflow.keras.layers import *
import tensorflow.keras.layers as layers

from tensorflow.keras.applications.xception import Xception


from sklearn.model_selection import train_test_split, StratifiedKFold

import PIL
from PIL import ImageOps, ImageFilter

#увеличим дефолтный размер графиков
from pylab import rcParams
rcParams['figure.figsize'] = 10, 5

#графики в svg выглядят более четкими
%config InlineBackend.figure_format = 'svg' 
%matplotlib inline

print(os.listdir("../input"))
print('Python       :', sys.version.split('\n')[0])
print('Numpy        :', np.__version__)
print('Tensorflow   :', tf.__version__)
print('Keras        :', tf.keras.__version__)

**Работаем с Tensorflow v2**

In [ ]:
!pip freeze > requirements.txt

# Основные настройки

In [ ]:
# В setup выносим основные настройки: так удобнее их перебирать в дальнейшем.

EPOCHS               = 10  # эпох на обучение
BATCH_SIZE           = 16 # уменьшаем batch если сеть большая, иначе не влезет в память на GPU
LR                   = 1e-4
VAL_SPLIT            = 0.15 # сколько данных выделяем на тест = 15%

CLASS_NUM            = 10  # количество классов в нашей задаче
IMG_SIZE             = 224 # какого размера подаем изображения в сеть
IMG_CHANNELS         = 3   # у RGB 3 канала
input_shape          = (IMG_SIZE, IMG_SIZE, IMG_CHANNELS)

DATA_PATH = '../input/'
PATH = "../working/car/" # рабочая директория

In [ ]:
# Устаналиваем конкретное значение random seed для воспроизводимости
os.makedirs(PATH,exist_ok=True)

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)  
PYTHONHASHSEED = 0

# EDA / Анализ данных

In [ ]:
train_df = pd.read_csv(DATA_PATH+"train.csv")
sample_submission = pd.read_csv(DATA_PATH+"sample-submission.csv")
train_df.head()

In [ ]:
train_df.info()

In [ ]:
sns.barplot(y=train_df.Category.value_counts().values,
           x=train_df.Category.value_counts().index,
           color='r')
# распределение классов достаточно равномерное - это хорошо

In [ ]:
print('Распаковываем картинки')
# Will unzip the files so that you can see them..
for data_zip in ['train.zip', 'test.zip']:
    with zipfile.ZipFile("../input/"+data_zip,"r") as z:
        z.extractall(PATH)
        
print(os.listdir(PATH))

In [ ]:
print('Пример картинок (random sample)')
plt.figure(figsize=(12,8))

random_image = train_df.sample(n=9)
random_image_paths = random_image['Id'].values
random_image_cat = random_image['Category'].values

In [ ]:
for index, path in enumerate(random_image_paths):
    im = PIL.Image.open(PATH+f'train/{random_image_cat[index]}/{path}')
    plt.subplot(3,3, index+1)
    plt.imshow(im)
    plt.title('Class: '+str(random_image_cat[index]))
    plt.axis('off')
plt.show()

Посмотрим на примеры картинок и их размеры чтоб понимать как их лучше обработать и сжимать.

In [ ]:
image = PIL.Image.open(PATH+'/train/0/100380.jpg')
imgplot = plt.imshow(image)
plt.show()
image.size

# Подготовка данных

## Аугментация данных
#### **Впилим новый аугментатор данных**

In [ ]:
! pip install git+https://github.com/mjkvaak/ImageDataAugmentor

In [ ]:
import tensorflow as tf
from ImageDataAugmentor.image_data_augmentor import *
import albumentations

In [ ]:

AUGMENTATIONS = albumentations.Compose([
    albumentations.HorizontalFlip(p=0.5),
    albumentations.Rotate(limit=30, interpolation=1, border_mode=4, value=None, mask_value=None, always_apply=False, p=0.5),
    albumentations.OneOf([
        albumentations.CenterCrop(height=224, width=200),
        albumentations.CenterCrop(height=200, width=224),
    ],p=0.5),
    albumentations.OneOf([
        albumentations.RandomBrightnessContrast(brightness_limit=0.3, contrast_limit=0.3),
        albumentations.RandomBrightnessContrast(brightness_limit=0.1, contrast_limit=0.1)
    ],p=0.5),
    albumentations.GaussianBlur(p=0.05),
    albumentations.HueSaturationValue(p=0.5),
    albumentations.RGBShift(p=0.5),
    albumentations.FancyPCA(alpha=0.1, always_apply=False, p=0.5),
    albumentations.Resize(240, 320)
])

# dataloaders
train_datagen_2 = ImageDataAugmentor(
        rescale=1./255,
        augment=AUGMENTATIONS,
        validation_split=VAL_SPLIT)

test_datagen = ImageDataGenerator(rescale=1. / 255)

# data generators
train_generator_2 = train_datagen_2.flow_from_directory(
        PATH+'train/',
        target_size=(IMG_SIZE, IMG_SIZE),
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        shuffle=True, 
        seed=RANDOM_SEED,
        subset='training')

test_generator_2 = train_datagen_2.flow_from_directory(
        PATH+'train/',
        target_size=(IMG_SIZE, IMG_SIZE),
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        shuffle=True, seed=RANDOM_SEED,
        subset='validation')

test_sub_generator = test_datagen.flow_from_dataframe( 
    dataframe=sample_submission,
    directory=PATH+'test_upload/',
    x_col="Id",
    y_col=None,
    shuffle=False,
    class_mode=None,
    seed=RANDOM_SEED,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,)

# Построение модели

Протеституем, как будет работать предобученная сеть EfficientNetB4

In [ ]:
!pip install efficientnet

In [ ]:
import efficientnet.tfkeras as efn

In [1]:
base_model = efn.EfficientNetB6(weights='imagenet', include_top=False, input_shape=input_shape)

NameError: name 'efn' is not defined

In [ ]:
base_model.summary()

In [ ]:
# Замораживаем веса в базовой модели
base_model.trainable = False

In [ ]:
# instantiating the model in the strategy scope creates the model on the TPU
#with tpu_strategy.scope():
# steps_per_execution=32

model2 = Model.Sequential()
model2.add(base_model)
# model2.add(layers.Conv2D(32, 3, activation = 'relu', padding = 'same', 
#                         input_shape = input_shape))
# model2.add(layers.BatchNormalization())

model2.add(layers.GlobalAveragePooling2D())
model2.add(layers.BatchNormalization()) 
#model2.add(layers.Dropout(0.25))
   
model2.add(layers.Dense(256, activation='relu'))
model2.add(layers.Dropout(0.25))

#model2.add(layers.BatchNormalization())
#     model2.add(layers.Dropout(0.25))

model2.add(layers.Dense(CLASS_NUM, activation='softmax'))

model2.compile(loss="categorical_crossentropy", 
               optimizer=optimizers.Adam(lr=LR), 
               metrics=["accuracy"],
               )

In [ ]:
# сколько слоев
print(len(model2.layers))

In [ ]:
# сколько слоев обучилось
print(len(model2.trainable_variables))

## Обучение модели

Добавим ModelCheckpoint чтоб сохранять прогресс обучения модели и можно было потом подгрузить и дообучить модель.

In [2]:

checkpoint = ModelCheckpoint('best_model.hdf5', 
                             monitor = ['val_accuracy'], 
                             verbose = 1, 
                             mode = 'max')

earlystop = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy',
                                             patience=3, 
                                             restore_best_weights=True)

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.25,
                              patience=2, min_lr=0.0000001, verbose=1,
                             mode='auto')

callbacks_list = [checkpoint,earlystop,reduce_lr] # lrscheduler,lr_callback

NameError: name 'ModelCheckpoint' is not defined

# Обучаем. Этап 1:

In [ ]:
history = model2.fit(
        train_generator_2,
        steps_per_epoch = train_generator_2.samples//train_generator_2.batch_size,
        validation_data = test_generator_2, 
        validation_steps = test_generator_2.samples//test_generator_2.batch_size,
        epochs = EPOCHS,
        callbacks = callbacks_list
)


In [ ]:
# сохраним итоговую сеть и подгрузим лучшую итерацию в обучении (best_model)
model2.save('../working/model_last.hdf5')
model2.load_weights('best_model.hdf5')

In [ ]:
scores = model2.evaluate_generator(test_generator_2, steps=len(test_generator_2), verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
 
epochs = range(len(acc))
 
plt.plot(epochs, acc, 'b', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
 
plt.figure()
 
plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
 
plt.show()

In [ ]:
round(model2.optimizer.lr.numpy(), 5)

# Обучение. Этап 2

In [ ]:
# начинаем постепенную разморозку слоев для базовой модели для обучения
base_model.trainable = True

# Fine-tune from this layer onwards
fine_tune_at = len(base_model.layers)//2 # количество слоев, которые будут обучаться

# Freeze all the layers before the `fine_tune_at` layer
for layer in base_model.layers[:fine_tune_at]:
  layer.trainable =  False

In [ ]:

model2.compile(loss="categorical_crossentropy", 
               optimizer=optimizers.Adam(lr=LR), 
               metrics=["accuracy"])

In [ ]:
history = model2.fit(
        train_generator_2,
        steps_per_epoch = train_generator_2.samples//train_generator_2.batch_size,
        validation_data = test_generator_2, 
        validation_steps = test_generator_2.samples//test_generator_2.batch_size,
        epochs = EPOCHS,
        callbacks = callbacks_list
)

In [ ]:
scores = model2.evaluate_generator(test_generator_2, steps=len(test_generator_2), verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
 
epochs = range(len(acc))
 
plt.plot(epochs, acc, 'b', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
 
plt.figure()
 
plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
 
plt.show()

# Обучение. Этап 3

In [ ]:
# # начинаем постепенную разморозку слоев для базовой модели для обучения
# base_model.trainable = True

# # Fine-tune from this layer onwards
# fine_tune_at = len(base_model.layers)//2 # количество слоев, которые будут обучаться
# fine_tune_at = int(fine_tune_at*1.5)

# # Freeze all the layers before the `fine_tune_at` layer
# for layer in base_model.layers[:fine_tune_at]:
#   layer.trainable =  False

# LR= 1e-5
# model2.compile(loss="categorical_crossentropy", 
#                optimizer=optimizers.Adam(lr=LR), 
#                metrics=["accuracy"])

In [ ]:
# history = model2.fit(
#         train_generator_2,
#         steps_per_epoch = len(train_generator_2),
#         validation_data = test_generator_2, 
#         validation_steps = len(test_generator_2),
#         epochs = EPOCHS,
#         callbacks = callbacks_list
# )

In [ ]:
# scores = model2.evaluate_generator(test_generator_2, steps=len(test_generator_2), verbose=1)
# print("Accuracy: %.2f%%" % (scores[1]*100))

In [ ]:
# acc = history.history['accuracy']
# val_acc = history.history['val_accuracy']
# loss = history.history['loss']
# val_loss = history.history['val_loss']
 
# epochs = range(len(acc))
 
# plt.plot(epochs, acc, 'b', label='Training acc')
# plt.plot(epochs, val_acc, 'r', label='Validation acc')
# plt.title('Training and validation accuracy')
# plt.legend()
 
# plt.figure()
 
# plt.plot(epochs, loss, 'b', label='Training loss')
# plt.plot(epochs, val_loss, 'r', label='Validation loss')
# plt.title('Training and validation loss')
# plt.legend()
 
# plt.show()

Пока пропускаем этап 3, так как особой пользы он не приносит.

# Обучение. Этап 4

In [ ]:
base_model.trainable = True

LR = 1e-5
model2.compile(loss="categorical_crossentropy", 
               optimizer=optimizers.Adam(lr=LR), 
               metrics=["accuracy"])

history = model2.fit(
        train_generator_2,
        steps_per_epoch = train_generator_2.samples//train_generator_2.batch_size,
        validation_data = test_generator_2, 
        validation_steps = test_generator_2.samples//test_generator_2.batch_size,
        epochs = EPOCHS,
        callbacks = callbacks_list
)

In [ ]:
scores = model2.evaluate_generator(test_generator_2, steps=len(test_generator_2), verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
 
epochs = range(len(acc))
 
plt.plot(epochs, acc, 'b', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
 
plt.figure()
 
plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
 
plt.show()

#  Обучение. Этап 5. Корректировка настроек

In [ ]:
EPOCHS               = 6
BATCH_SIZE           = 4 # уменьшаем batch если сеть большая, иначе не влезет в память на GPU
LR                   = 1e-5

IMG_SIZE             = 512 # увеличиваем размер картинки
IMG_CHANNELS         = 3
input_shape          = (IMG_SIZE, IMG_SIZE, IMG_CHANNELS)

In [ ]:
AUGMENTATIONS = albumentations.Compose([
    albumentations.HorizontalFlip(p=0.5),
    albumentations.Rotate(limit=30, interpolation=1, border_mode=4, value=None, mask_value=None, always_apply=False, p=0.5),

])

# dataloaders
train_datagen_2 = ImageDataAugmentor(
        rescale=1./255,
        augment=AUGMENTATIONS,
        validation_split=VAL_SPLIT)

test_datagen = ImageDataGenerator(rescale=1. / 255)

In [ ]:
# data generators
train_generator_2 = train_datagen_2.flow_from_directory(
        PATH+'train/',
        target_size=(IMG_SIZE, IMG_SIZE),
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        shuffle=True, 
        seed=RANDOM_SEED,
        subset='training')

test_generator_2 = train_datagen_2.flow_from_directory(
        PATH+'train/',
        target_size=(IMG_SIZE, IMG_SIZE),
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        shuffle=True, 
        seed=RANDOM_SEED,
        subset='validation')

test_sub_generator = test_datagen.flow_from_dataframe( 
    dataframe=sample_submission,
    directory=PATH+'test_upload/',
    x_col="Id",
    y_col=None,
    shuffle=False,
    class_mode=None,
    seed=RANDOM_SEED,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,)

In [ ]:
# загружаем базовую модель

base_model = efn.EfficientNetB6(weights='imagenet', include_top=False, input_shape=input_shape)

# # Устанавливаем новую "голову" (head)

model2 = Model.Sequential()
model2.add(base_model)
# model2.add(layers.Conv2D(32, 3, activation = 'relu', padding = 'same', 
#                         input_shape = input_shape))
# model2.add(layers.BatchNormalization())

model2.add(layers.GlobalAveragePooling2D())
model2.add(layers.BatchNormalization())
#model2.add(layers.Dropout(0.25))    

model2.add(layers.Dense(256, activation='relu'))
model2.add(layers.Dropout(0.25))
#model2.add(layers.BatchNormalization())
#     model2.add(layers.Dropout(0.25))

model2.add(layers.Dense(CLASS_NUM, activation='softmax'))

model2.compile(loss="categorical_crossentropy", 
               optimizer=optimizers.Adam(lr=LR), 
               metrics=["accuracy"],
               )

In [ ]:
# Загружаем веса уже обученной модели
model2.load_weights('best_model.hdf5')

In [ ]:
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.3,
                              patience=2, min_lr=0.0000001, verbose=1,
                             mode='auto')

callbacks_list = [checkpoint,earlystop,reduce_lr]

In [ ]:

history = model2.fit(
        train_generator_2,
        steps_per_epoch = train_generator_2.samples//train_generator_2.batch_size,
        validation_data = test_generator_2, 
        validation_steps = test_generator_2.samples//test_generator_2.batch_size,
        epochs = EPOCHS,
        callbacks = callbacks_list
)

In [ ]:
scores = model2.evaluate_generator(test_generator_2, steps=len(test_generator_2), verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
 
epochs = range(len(acc))
 
plt.plot(epochs, acc, 'b', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
 
plt.figure()
 
plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
 
plt.show()

# Предсказание на тестовых данных

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
test_sub_generator.samples

In [ ]:
test_sub_generator.reset()
predictions = model2.predict_generator(test_sub_generator, steps=len(test_sub_generator), verbose=1) 
predictions = np.argmax(predictions, axis=-1) #multiple categories
label_map = (train_generator_2.class_indices)
label_map = dict((v,k) for k,v in label_map.items()) #flip k,v
predictions = [label_map[k] for k in predictions]

In [ ]:
filenames_with_dir=test_sub_generator.filenames
submission = pd.DataFrame({'Id':filenames_with_dir, 'Category':predictions}, columns=['Id', 'Category'])
submission['Id'] = submission['Id'].replace('test_upload/','')
submission.to_csv(PATH+'submission.csv', index=False)
print('Save submit')


In [ ]:
submission.to_csv('submission.csv', index=False)

In [ ]:
submission.head()

# TTA

In [ ]:
model2.load_weights('best_model.hdf5')

In [ ]:

AUGMENTATIONS = albumentations.Compose([
    albumentations.HorizontalFlip(p=0.5),
    albumentations.Rotate(limit=30, interpolation=1, border_mode=4, value=None, mask_value=None, always_apply=False, p=0.5),
    albumentations.OneOf([
        albumentations.CenterCrop(height=224, width=200),
        albumentations.CenterCrop(height=200, width=224),
    ],p=0.5),
    albumentations.OneOf([
        albumentations.RandomBrightnessContrast(brightness_limit=0.3, contrast_limit=0.3),
        albumentations.RandomBrightnessContrast(brightness_limit=0.1, contrast_limit=0.1)
    ],p=0.5),
    albumentations.GaussianBlur(p=0.05),
    albumentations.HueSaturationValue(p=0.5),
    albumentations.RGBShift(p=0.5),
    albumentations.FancyPCA(alpha=0.1, always_apply=False, p=0.5),
    albumentations.Resize(IMG_SIZE, IMG_SIZE)
])

# dataloaders
test_datagen = ImageDataAugmentor(
        rescale=1./255,
        augment=AUGMENTATIONS,
        validation_split=VAL_SPLIT)


test_sub_generator = test_datagen.flow_from_dataframe( 
    dataframe=sample_submission,
    directory=PATH+'test_upload/',
    x_col="Id",
    y_col=None,
    shuffle=False,
    class_mode=None,
    seed=RANDOM_SEED,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,)

In [ ]:
tta_steps = 10
predictions = []

for i in range(tta_steps):
    preds = model2.predict_generator(test_sub_generator, verbose=1) 
    predictions.append(preds)

pred = np.mean(predictions, axis=0)

In [ ]:
predictions = np.argmax(pred, axis=-1) #multiple categories
label_map = (train_generator_2.class_indices)
label_map = dict((v,k) for k,v in label_map.items()) #flip k,v
predictions = [label_map[k] for k in predictions]

filenames_with_dir=test_sub_generator.filenames
TTA_submission = pd.DataFrame({'Id':filenames_with_dir, 'Category':predictions}, columns=['Id', 'Category'])
TTA_submission['Id'] = TTA_submission['Id'].replace('test_upload/','')
TTA_submission.to_csv('TTA_submission.csv', index=False)
print('Save submit')

In [ ]:
TTA_submission.head(5)

In [ ]:
# Clean PATH
# import shutil
# shutil.rmtree(PATH)